In [ ]:
!pip install statsbombpy

In [29]:
from statsbombpy import sb
import numpy as np
import pandas as pd

In [ ]:
# Download the data ----

matches = sb.matches(competition_id=2, season_id=27)

events = pd.concat([sb.events(match_id=x) for x in matches['match_id']])

In [131]:
# Calculate summary metrics for each possession (takes a minute)
possession_summary = (
    events
      .sort_values(['match_id', 'index'])
      .loc[lambda x: x['type'].isin(['Pass', 'Carry', 'Shot'])]
      .assign(
          loc_x=lambda x: [y[0] if isinstance(y, list) else np.nan for y in x['location']],
          time=lambda x: 60 * x['minute'] + x['second']
      )
      .groupby(['match_id', 'possession', 'team'])
      .agg(
          start_x=('loc_x', 'first'),
          end_x=('loc_x', 'last'),
          start_time=('time', 'first'),
          end_time=('time', 'last'),
          num_passes=('type', lambda x: (x == 'Pass').sum()),
          num_shots=('type', lambda x: (x == 'Shot').sum()),
          num_goals=('shot_outcome', lambda x: (x == 'Goal').sum())
      )
      .assign(
          distance = lambda x: x['end_x'] - x['start_x'],
          duration = lambda x: x['end_time'] - x['start_time'],
          speed = lambda x: x['distance'] / (x['duration'] + 0.01)
      )
      .reset_index()
)

In [124]:
# Aggregate possession outcomes according to possession length
possession_by_num_passes = (
    possession_summary
      .groupby('num_passes')
      .agg(
          count=('num_passes', 'size'),
          prob_shot=('num_shots', lambda x: (x > 0).mean()),
          prob_goal=('num_goals', lambda x: (x > 0).mean())
      )
      .reset_index()
      .loc[lambda x: x['num_passes'] > 0]
)

In [ ]:
# Plot goal probability versus possession length ----

import matplotlib.pyplot as plt

plt_data = possession_by_num_passes[lambda x: x['num_passes'] < 20]
plt.scatter(x=plt_data['num_passes'], y=plt_data['prob_goal'])
plt.show()

# EXERCISE #1: What does this figure look like if you filter out all
#   possessions starting in the opponent's half of the field?

In [ ]:
# EXERCISE #2: To visualize team styles, create a plot of average possession
#   "speed" (as defined above) against average number of passes per possession.
#   Only include possessions that last longer than 5 seconds.
#   Does Leicester City stand out?

In [ ]:
# Investigate pressing ----

# A tactic we discussed both Pep Guardiola and Jurgen Klopp emphasizing is
# trying to win back the ball quickly after losing possession in open play.

# In this block of code, we identify turnovers that happen in play (as opposed
# to turnovers that lead to set pieces), and we calculate the time until the
# next turnover happens (the next turnover may not necessarily happen in play).
turnover_data = (
    events
      .assign(time=lambda x: 60 * x['minute'] + x['second'])
      .loc[lambda x: x['type'].isin(['Pass', 'Carry', 'Shot'])]
      .assign(
          is_turnover_in_play=lambda x:
            (x['possession_team_id'] != x['possession_team_id'].shift(-1))
            & (x['play_pattern'].shift(-1) == 'Regular Play'),  # in play
          is_turnover=lambda x:
            (x['possession_team_id'] != x['possession_team_id'].shift(-1)),
          turnover_time=lambda x: np.where(x['is_turnover'], x['time'] + x['duration'], np.nan),
          next_turnover_time=lambda x: x['turnover_time'].fillna(method='bfill').shift(-1),
          # Need to guarantee next turnover is in same match and period
          turnover_match_id=lambda x: np.where(x['is_turnover'], x['match_id'], np.nan),
          next_turnover_match_id=lambda x: x['turnover_match_id'].fillna(method='bfill').shift(-1),
          turnover_period=lambda x: np.where(x['is_turnover'], x['period'], np.nan),
          next_turnover_period=lambda x: x['turnover_period'].fillna(method='bfill').shift(-1),
          # Calculate whether the next turnover happens in under six seconds
          under_six_seconds=lambda x:
            (x['next_turnover_time'] - x['turnover_time'] < 6)
            & (x['match_id'] == x['next_turnover_match_id'])
            & (x['period'] == x['next_turnover_period'])
      )
      .loc[lambda x: x['is_turnover_in_play']]
      [['team', 'turnover_time', 'next_turnover_time', 'under_six_seconds']]
)

print(turnover_data)

# EXERCISE #3: Calculate a leaderboard of the frequency with which each team
#   recovered possession within 6 seconds of committing a turnover.